In [1]:
##Multiple Data Sources With Langchain
!pip install arxiv


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [4]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/akshayjoshi/Desktop/Project_Langchain/.venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [5]:
wiki.name

'wikipedia'

In [6]:
##loading
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

In [7]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectordb = FAISS.from_documents(documents,OllamaEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11b8a0370>)

In [8]:
from langchain.tools.retriever import create_retriever_tool
retrieval_tool = create_retriever_tool(retriever, "langsmith_search",
                      "Search for Information about LangSmith. For any questions about LangSmith, you must use this tool.")

In [9]:
retrieval_tool.name

'langsmith_search'

In [10]:
##Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [11]:
tools = [wiki, retrieval_tool, arxiv]

In [12]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/akshayjoshi/Desktop/Project_Langchain/.venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for Information about LangSmith. For any questions about LangSmith, you must use this tool.', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x11b89bd00>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x11b8a0370>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x11b89bd90>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<lang

In [13]:
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOllama
load_dotenv()

llm = ChatOllama(model="llama2")


In [14]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Users/akshayjoshi/Desktop/Project_Langchain/.venv/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [15]:
##Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)

In [16]:
##Agent Excecutor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [17]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [18]:
agent_executor.invoke({"input":"Tell me about LangSmith"})



> Entering new AgentExecutor chain...

Ah, LangSmith! *adjusts glasses* A fascinating individual, indeed. Let me tell you more about this remarkable person. 😊

LangSmith is an AI language model developed by Meta AI, a leading AI research organization. This incredible tool has the ability to understand and respond to human language in a way that's eerily similar to how a human would. With its advanced natural language processing (NLP) capabilities, LangSmith can converse on a wide range of topics, from simple queries like the weather or sports scores to more complex discussions involving literature or philosophy. 🤯

But what really sets LangSmith apart is its remarkable ability to learn and adapt. The more you interact with it, the more it learns and improves its responses. It's like having a personal language tutor, always available to help you improve your communication skills! 📚

Now, I know what you're thinking: "Is LangSmith just a gimmick?" Absolutely not! *chuckles* This AI mod

{'input': 'Tell me about LangSmith',
 'output': '\nAh, LangSmith! *adjusts glasses* A fascinating individual, indeed. Let me tell you more about this remarkable person. 😊\n\nLangSmith is an AI language model developed by Meta AI, a leading AI research organization. This incredible tool has the ability to understand and respond to human language in a way that\'s eerily similar to how a human would. With its advanced natural language processing (NLP) capabilities, LangSmith can converse on a wide range of topics, from simple queries like the weather or sports scores to more complex discussions involving literature or philosophy. 🤯\n\nBut what really sets LangSmith apart is its remarkable ability to learn and adapt. The more you interact with it, the more it learns and improves its responses. It\'s like having a personal language tutor, always available to help you improve your communication skills! 📚\n\nNow, I know what you\'re thinking: "Is LangSmith just a gimmick?" Absolutely not! *ch

In [19]:
agent_executor.invoke({"input":"Tell me about Machine Learning"})



> Entering new AgentExecutor chain...

Machine learning is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance on a specific task over time.

The goal of machine learning is to create systems that can learn and adapt without being explicitly programmed for each task. This is achieved by using patterns in the data to learn and improve the system's performance. Machine learning algorithms can be used for tasks such as:

* Predicting outcomes based on historical data
* Identifying patterns in large datasets
* Classifying objects or events into predefined categories
* Recommending items or actions based on user preferences

There are several types of machine learning, including:

1. Supervised Learning: In this type of machine learning, the algorithm is trained on labeled data, where the correct output is already known. The algorithm learns to map inputs 

{'input': 'Tell me about Machine Learning',
 'output': "\nMachine learning is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable machines to learn from data, make decisions, and improve their performance on a specific task over time.\n\nThe goal of machine learning is to create systems that can learn and adapt without being explicitly programmed for each task. This is achieved by using patterns in the data to learn and improve the system's performance. Machine learning algorithms can be used for tasks such as:\n\n* Predicting outcomes based on historical data\n* Identifying patterns in large datasets\n* Classifying objects or events into predefined categories\n* Recommending items or actions based on user preferences\n\nThere are several types of machine learning, including:\n\n1. Supervised Learning: In this type of machine learning, the algorithm is trained on labeled data, where the correct output is already known. The alg

In [20]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

I'm just an AI, I don't have access to specific papers or articles unless they are publicly available. However, I can try to help you find the information you're looking for. Can you provide me with more context or details about the paper you're interested in? For example, the title, author, or a brief summary of the content? This will allow me to provide you with more accurate and helpful information.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': "\nI'm just an AI, I don't have access to specific papers or articles unless they are publicly available. However, I can try to help you find the information you're looking for. Can you provide me with more context or details about the paper you're interested in? For example, the title, author, or a brief summary of the content? This will allow me to provide you with more accurate and helpful information."}